In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.tabular.all import load_learner

In [2]:
learn = load_learner('titanic_model_v3.3.pkl')
test_df = pd.read_csv('test.csv')
test_df.head()

c:\Code\Local Code\titanic_survive\.venv\Lib\site-packages\fastai\learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
median_fare = test_df['Fare'].median()
test_df['Fare'] = test_df['Fare'].fillna(median_fare)

In [4]:
test_dl = learn.dls.test_dl(test_df)
probs, _ = learn.get_preds(dl=test_dl)

c:\Code\Local Code\titanic_survive\.venv\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [5]:
pred_labels = (probs>0.5).long().view(-1)
print(pred_labels[:5])
check = pred_labels.sum()
print("Predicted survivors:", pred_labels.sum().item())

tensor([0, 0, 0, 0, 0])
Predicted survivors: 136


In [8]:
test_df['Survived']   = pred_labels.numpy()
test_df[['PassengerId','Survived']].to_csv('submission.csv', index=False)